In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
from sklearn.preprocessing import MinMaxScaler

        
        
import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.dataset import random_split

/kaggle/input/lish-moa/test_features.csv
/kaggle/input/lish-moa/sample_submission.csv
/kaggle/input/lish-moa/train_features.csv
/kaggle/input/lish-moa/train_targets_scored.csv
/kaggle/input/lish-moa/train_targets_nonscored.csv


In [2]:
train_features = pd.read_csv('../input/lish-moa/train_features.csv')
train_targets_scored = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
train_targets_nonscored = pd.read_csv('../input/lish-moa/train_targets_nonscored.csv')
test_features = pd.read_csv('../input/lish-moa/test_features.csv')
submission = pd.read_csv('../input/lish-moa/sample_submission.csv')

In [5]:
def convert_categorical(category, df):
    s = str(category)
    df = pd.get_dummies(df, columns=[s])
    return df

def encode_df(df):
    
    df_encoded = convert_categorical('cp_type', df)
    df_encoded1 = convert_categorical('cp_dose', df_encoded)
    return(df_encoded1)

train_features_encoded = encode_df(train_features)

if 'cp_dose_D1'and'cp_type_trt_cp' in train_features_encoded.columns:
    print("encoded columns successfully")
else:
    print("Nope")

encoded columns successfully


In [6]:
train_features_encoded.shape, train_targets_scored.shape


((23814, 878), (23814, 207))

In [7]:
feature_columns = train_features_encoded.columns[1:]
target_columns = train_targets_scored.columns[1:]

In [8]:
train_cat = train_features_encoded.merge(train_targets_scored, on='sig_id')


In [9]:
dummy_df = train_cat.loc[:, train_cat.columns != 'sig_id']
df_float = dummy_df.astype(float)
scaler = MinMaxScaler()
df_float_scaled = pd.DataFrame(scaler.fit_transform(df_float), columns = df_float.columns)
df_float_scaled['sig_id'] = train_features_encoded['sig_id']
df_float_scaled.info()


NameError: name 'df_encoded' is not defined

In [10]:
class TrainDataset(Dataset):
    def __init__(self, df, feature_columns, target_columns):
        
        self.features  = df[feature_columns].values
        self.targets = df[target_columns].values
        
    def __len__(self):
        return self.features.shape[0]

    def __getitem__(self, idx):
        feature = torch.tensor(self.features[idx])
        target = torch.tensor(self.target[idx])
        
        return feature,target

In [ ]:
full_dataset = TrainDataset(df_float_scaled, feature_columns, target_columns)

In [ ]:
train_size = int(0.9 * len(full_dataset))  ## 90/10 split
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True, num_workers = 8)

val_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle = True, num_workers = 8)

print(len(train_loader), "batches ")
print(len(val_loader), " batches ")

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()

        self.dropout = nn.Dropout(p=0.2)
        self.fc1 = nn.Linear(5, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 128)
        self.fc4 = nn.Linear(128, 128)

        self.fc5 = nn.Linear(128, 64)
        self.fc6 = nn.Linear(64, 32)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.dropout(F.relu(self.fc3(x)))
        x = self.dropout(F.relu(self.fc4(x)))

        x = F.relu(self.fc5(x))

        return (self.fc6(x))
    
model = Model()
print(model)
model = model.cuda()